In [21]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import r2_score

In [28]:
df_train = pd.read_csv('train.csv')
X = df_train[['OverallQual', 'GrLivArea', 'GarageCars']]
y = df_train['SalePrice']

df_test = pd.read_csv('test.csv')
X_test = df_test[['OverallQual', 'GrLivArea', 'GarageCars']]
# y_test = df_test[['SalePrice']]

In [8]:
### prebuild random forest-just one tree
reg = RandomForestRegressor(
  n_estimators=1, 
  max_depth=2, 
  bootstrap=False, 
  random_state=False
)
reg.fit(X, y)

RandomForestRegressor(bootstrap=False, max_depth=2, n_estimators=1,
                      random_state=False)

In [10]:
class DecisionTreeRegressor:
  
  def fit(self, X, y, min_leaf = 5):
    self.dtree = Node(X, y, np.array(np.arange(len(y))), min_leaf)
    return self
  
  def predict(self, X):
    return self.dtree.predict(X.values)

In [11]:
class Node:
    def __init__(self, x, y, idxs, min_leaf=5):
        self.x = x 
        self.y = y
        self.idxs = idxs 
        self.min_leaf = min_leaf
        self.row_count = len(idxs)
        self.col_count = x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
    def find_varsplit(self): #find where to split
        for c in range(self.col_count): self.find_better_split(c)
        if self.is_leaf: return
        x = self.split_col
        lhs = np.nonzero(x <= self.split)[0]
        rhs = np.nonzero(x > self.split)[0]
        self.lhs = Node(self.x, self.y, self.idxs[lhs], self.min_leaf)
        self.rhs = Node(self.x, self.y, self.idxs[rhs], self.min_leaf)
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]

    @property
    def is_leaf(self): return self.score == float('inf') 
    def find_better_split(self, var_idx):
        x = self.x.values[self.idxs, var_idx]

        for r in range(self.row_count):
            lhs = x <= x[r]
            rhs = x > x[r]
            if rhs.sum() < self.min_leaf or lhs.sum() < self.min_leaf: continue

            curr_score = self.find_score(lhs, rhs)
            if curr_score < self.score: 
                self.var_idx = var_idx
                self.score = curr_score
                self.split = x[r]

    def find_score(self, lhs, rhs):
        y = self.y[self.idxs]
        lhs_std = y[lhs].std()
        rhs_std = y[rhs].std()
        return lhs_std * lhs.sum() + rhs_std * rhs.sum()
    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        node = self.lhs if xi[self.var_idx] <= self.split else self.rhs
        return node.predict_row(xi)

In [14]:
regressor = DecisionTreeRegressor().fit(X, y)
preds = regressor.predict(X)

In [23]:
r2_score(y, preds)

0.8504381072711565

In [26]:
pred_test = regressor.predict(X_test)
r2_score(y_test, pred_test)

NameError: name 'y_test' is not defined

In [ ]:
def rmse(h, y):
  return sqrt(mean_squared_error(h, y))